# K-nearest Neighbors

## 1. Definition
KNN is a non-parametric, instance-based learning algorithm used for classification and regression. It predicts the label/value of a new point by finding the k closest data points in the training set and aggregating their labels. It solves the problem of modeling complex, non-linear decision boundaries without making strong assumptions about the underlying data distribution. It's a lazy learner - it doesn't build a model during training but memorizes the dataset and makes decisions only during inference.

## 2. Core Idea
KNN relies on the smoothness assumption - points that are close to each other in the feature space likely belong to the same class or have similar target values.
Decision boundary is not extremely complex (KNN can struggle in high-dimensional spaces → curse of dimensionality).
* **Feature Scaling is Crucial:** Since it relies on distance calculation, it implicitly assumes that all features contribute equally to the distance, which makes **feature scaling (normalization or standardization)** essential to prevent features with larger magnitudes from dominating the distance calculation.

## 3. Mechanism
The workflow is unique because there is effectively no "training" phase.
* Storage: The model simply stores the entire training dataset in memory.
* Distance Calculation: When a new query point $x_i$ arrives, KNN calculates the distance (e.g. Euclidean) between it and every other points.
* Neighbor Retrieval: It identifies the K points with the smallest distance.
* Voting/Averaging:
    * Classification: KNN assigns the class that is the **most frequent** among its $k$ nearest neighbors (a majority vote).
    * Regression: KNN predicts the value by calculating the **average** (or weighted average) of the target values of its $k$ nearest neighbors.

## 4. Mathematics and Training
* Loss function: None explicitly optimized during training
* Distance Metrics:
    * Euclidean ($L2$): $\sqrt{\sum{(x_i - y_i)^2}}$
    * Manhattan ($L1$): $\sum{|x_i-y_i|}$
    * Minkowski: Generalization of L1 and L2
* K: We use cross-validation to choose k
	* Small K → high variance, low bias
	* Large K → low variance, high bias

## 5. Pros and Cons
* Pros:
    * Zero training time O(1)
    * Simple & powerful: can learn very complex non-linear boundaries
    * Adaptability: new training data can be added instantly without retraining a model
* Cons:
    * Slow Inference: Querying is expensive O(N) .. compare against every data point
    * Memory Expensive
    * Feature Scaling: It is broken by unscaled data
    * High-dimensional data kills performance: In high dimensions (e.g., >100 features), "distance" becomes meaningless because all points become roughly equidistant.
    * Imbalanced data has bias


## 6. Production System
Almost never used in production due to latency.


## 7. Other Variants
* Approximate Nearest Neighbors (ANN): These trade 1% accuracy for 1000x speedups by searching "buckets" of points rather than every single point.
* Indexing: Use tree structure like KD-tree or Ball trees to speed up search from O(N) to O(log N)

In [1]:
import sys, os
root = os.path.abspath("..")
sys.path.append(root)

from src.knn import KNN
import pandas as pd
import numpy as np

## Case 1: Recommender System

* Amazon used “item-to-item similarity” (a KNN-style algorithm)
* Google Search used KNN for early query similarity and document clustering
* YouTube used KNN for item similarity before the “Deep Neural Recommender” era


### Business Problem
Amazon has millions of items but each user interacts with only a few.
We need recommendations that are fast, scalable, personalizable, and easy to update incrementally.

Classic user-based content filtering breaks when:
* Each user has very few interactions
* User similarities are sparse
* The user base is huge


### Data Problem
We use 'item-item' KNN to resolve this issue. The goal is to build a system that:
* Compute similarity between items
* Retrieves the top-k similar items for any item
* Uses these KNN results to generate user-level recommendations in milliseconds


### Dataset
Our dataset contains the user purchase history.


### Feature Engineering
* Encode user-item interactions: Each item becomes a vector: 
    - iPad -> [1,0,1,...]
    - iPhone -> [1,1,0,...]
* Compute KNN (item similarity): Use vector distances e.g. Cosine similarity, Jaccard, Person; 
    - For each item $i$, we store: top-k most similar items e.g. "KNN for iPhone" → [AirPods, iPad, Macbook]

* Real-time user recommendation
    - Look at items they interacted with
    - Fetch precomputed item-similarity list
    - Combine/weight scores
    - Return top-ranked items
This is KNN applied on items only → extremely scalable.



In [2]:
# Dummy purchase history: (user, item)
data = [
    ("U1", "iPhone 15"),
    ("U1", "AirPods Pro"),
    ("U1", "iPad"),

    ("U2", "iPhone 15"),
    ("U2", "AirPods Pro"),

    ("U3", "MacBook Air"),
    ("U3", "iPhone 15"),
    ("U3", "AirPods Pro"),

    ("U4", "MacBook Air"),
    ("U4", "iPad"),

    ("U5", "Kindle Paperwhite"),
    ("U5", "Echo Dot"),

    ("U6", "Kindle Paperwhite"),
    ("U6", "iPad"),

    ("U7", "Echo Dot"),
    ("U7", "iPhone 15"),

    ("U8", "Echo Dot"),
    ("U8", "AirPods Pro"),
]

df = pd.DataFrame(data, columns=["user_id", "item_name"])
df.head()

,user_id,item_name
0,U1,iPhone 15
1,U1,AirPods Pro
2,U1,iPad
3,U2,iPhone 15
4,U2,AirPods Pro


In [3]:
## Build user-item matrix
user_item_df = (
    df
    .assign(purchased=1)
    .pivot_table(index="user_id", columns="item_name", values="purchased", fill_value=0)
)
user_item_df

item_name,AirPods Pro,Echo Dot,Kindle Paperwhite,MacBook Air,iPad,iPhone 15
user_id,,,,,,
U1,1.0,0.0,0.0,0.0,1.0,1.0
U2,1.0,0.0,0.0,0.0,0.0,1.0
U3,1.0,0.0,0.0,1.0,0.0,1.0
U4,0.0,0.0,0.0,1.0,1.0,0.0
U5,0.0,1.0,1.0,0.0,0.0,0.0
U6,0.0,0.0,1.0,0.0,1.0,0.0
U7,0.0,1.0,0.0,0.0,0.0,1.0
U8,1.0,1.0,0.0,0.0,0.0,0.0


In [4]:
# Train KNN and store results
item_names = user_item_df.columns.tolist()
X_items = user_item_df.values.T
y_items = np.arange(X_items.shape[0])

knn_items= KNN(k=3, distance_metric="cosine")
knn_items.fit(X_items, y_items)

Training KNN with k=3: Storing 6 samples.


In [5]:
# Find similar items with your KNN
def get_similar_items_knn(target_item_name, k=3):
    if target_item_name not in item_names:
        raise ValueError(f"{target_item_name} not found.")
    
    # index of the item
    item_idx = item_names.index(target_item_name)
    item_vec = X_items[item_idx]

    # query k+1 neighbors because the closest will be itself (distance 0)
    distances, indices = knn_items.kneighbors(item_vec, k=k+1, return_distance=True)
    distances = distances[0]
    indices = indices[0]

    # Drop itsef (distance 0, index = item_idx)
    mask = indices != item_idx
    neighbor_indices = indices[mask][:k]
    neighbor_distances = distances[mask][:k]

    results = []
    for d, idx in zip(neighbor_distances, neighbor_indices):
        results.append((item_names[idx], float(d)))
    return results

print("Similar to 'iPhone 15':")
print(get_similar_items_knn("iPhone 15", k=3))

print("\nSimilar to 'Kindle Paperwhite':")
print(get_similar_items_knn("Kindle Paperwhite", k=3))

Similar to 'iPhone 15':
[('AirPods Pro', 0.25), ('MacBook Air', 0.6464466094067263), ('Echo Dot', 0.7113248654051871)]

Similar to 'Kindle Paperwhite':
[('Echo Dot', 0.5917517095361371), ('iPad', 0.5917517095361371), ('AirPods Pro', 1.0)]


In [6]:
# Recommend items for a user using item-KNN

def recommend_for_user_knn(user_id, k_item=3, top_n=5):
    """
    Recommend items for a given user based on item-to-item KNN
    using the KNN model built on item vectors.
    """
    if user_id not in user_item_df.index:
        raise ValueError(f"Unknown user_id: {user_id}")

    user_row = user_item_df.loc[user_id]
    purchased_items = user_row[user_row > 0].index.tolist()

    if not purchased_items:
        return []  # cold start

    scores = {}

    for item in purchased_items:
        neighbors = get_similar_items_knn(item, k=k_item)
        for neigh_item, dist in neighbors:
            if neigh_item in purchased_items:
                continue
            # Convert distance -> similarity-like score (smaller dist = higher score)
            score = 1.0 / (1.0 + dist)
            scores[neigh_item] = scores.get(neigh_item, 0.0) + score

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked[:top_n]


print("Recommendations for U1:")
print(recommend_for_user_knn("U1", k_item=3, top_n=5))

print("\nRecommendations for U5:")
print(recommend_for_user_knn("U5", k_item=3, top_n=5))

Recommendations for U1:
[('MacBook Air', 1.8429759183234267), ('Echo Dot', 1.168685175112245), ('Kindle Paperwhite', 0.6282386844688338)]

Recommendations for U5:
[('AirPods Pro', 1.0843425875561223), ('iPad', 0.6282386844688338), ('iPhone 15', 0.5843425875561225)]


### Why do we favor cosine distance over euclidean distance?

In item-to-item recommenders, cosine distance is preferred over Euclidean because it normalizes for popularity. Cosine measures similarity in the pattern of users who bought an item, regardless of how many purchases it had. Euclidean distance is dominated by vector magnitude and performs poorly in sparse, high-dimensional, binary user–item matrices. Cosine gives much more meaningful item similarity for recommender systems.

1. Cosine distance measures direction, not magnitude:

* Euclidean distance is sensitive to vector length:
	- Items purchased by many users have longer vectors
	- Items purchased by few users have shorter vectors

* Example
	-	Item A bought by 10,000 users
	-	Item B bought by 200 users
	-	Item C bought by 205 users, mostly overlapping with Item B

Euclidean says A is closer to C than B, because A’s vector is huge.
Cosine says B is closest to C, because they share the same pattern of buyers, even if fewer.


2. Cosine similarity handles sparse, high-dimensional, binary data
In sparse spaces:
	- Euclidean distance mostly measures “how many zeros don’t overlap”
	- Cosine measures angle → pattern similarity despite sparsity

3. Cosine is scale-invariant 

Cosine similarity:

$\cos(\theta) = \frac{A \cdot B}{\|A\| \, \|B\|}$

This normalizes both vectors.

Meaning:
	•	If 100 people buy iPhone → long vector
	•	If 10 people buy iPhone → short vector
	•	Cosine will treat them as equally directional


4. Cosine gives more intuitive “similar items” in CF
Cosine = “same users?”
Euclidean = “same number of users bought them?”

The second is not what we want.

